In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import boto3
import logging
from datetime import datetime

boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

<IPython.core.display.Javascript object>

In [3]:
import sagemaker
from sagemaker.transformer import Transformer

<IPython.core.display.Javascript object>

In [4]:
sgmkr_clnt = boto3.client("sagemaker")
sgmkr_rt = boto3.client("runtime.sagemaker")

<IPython.core.display.Javascript object>

In [5]:
# role_arn = sagemaker.get_execution_role()
role_arn = os.getenv("SGMKR_ROLE_ARN")

<IPython.core.display.Javascript object>

#### Create model

In [6]:
model_img = "544295431143.dkr.ecr.ap-southeast-2.amazonaws.com/xgboost:latest"

<IPython.core.display.Javascript object>

In [7]:
model_path = (
    "s3://sgmkr-course/iris/model/iris-xgboost-2023-02-03-06-03-15/output/model.tar.gz"
)

<IPython.core.display.Javascript object>

In [8]:
model_name = "iris-xgboost-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(model_name)

iris-xgboost-2023-03-22-19-14-35


<IPython.core.display.Javascript object>

In [9]:
response = sgmkr_clnt.create_model(
    ModelName=model_name,
    PrimaryContainer={"Image": model_img, "ModelDataUrl": model_path},
    ExecutionRoleArn=role_arn,
)

print(response)

{'ModelArn': 'arn:aws:sagemaker:ap-southeast-2:042990164028:model/iris-xgboost-2023-03-22-19-14-35', 'ResponseMetadata': {'RequestId': 'eabdc9b6-c949-4dee-a498-67f728564b77', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'eabdc9b6-c949-4dee-a498-67f728564b77', 'content-type': 'application/x-amz-json-1.1', 'content-length': '99', 'date': 'Wed, 22 Mar 2023 08:14:38 GMT'}, 'RetryAttempts': 0}}


<IPython.core.display.Javascript object>

In [10]:
bucket = "sgmkr-course"
prefix = "iris"

<IPython.core.display.Javascript object>

#### Batch Transform

In [24]:
batch_ip = 's3://{}/{}/{}'.format(bucket, prefix, 'batch_transform')
batch_op = 's3://{}/{}/{}'.format(bucket, prefix, 'batch_transform')

<IPython.core.display.Javascript object>

In [28]:
transformer = Transformer(
    model_name=model_name,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=batch_op,
)

transformer.transform(
    data=batch_ip, data_type="S3Prefix", content_type="text/csv"
)
transformer.wait()

.................................
Arguments: serve
[2023-02-03 09:53:36 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2023-02-03 09:53:36 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2023-02-03 09:53:36 +0000] [1] [INFO] Using worker: gevent
[2023-02-03 09:53:36 +0000] [21] [INFO] Booting worker with pid: 21
[2023-02-03 09:53:36 +0000] [22] [INFO] Booting worker with pid: 22
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util.ssl_ (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.py)', 'urllib3.util (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py)']. 
  monkey.patch_all(subprocess=True)
[2023-02

UnexpectedStatusException: Error for Transform job xgboost-2023-02-03-09-48-15-211: Failed. Reason: ClientError: See job logs for more information

<IPython.core.display.Javascript object>

#### Endpoint

In [11]:
ep_config_name = "tmp-ep-config-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_config_name)

tmp-ep-config-2023-03-22-19-15-04-118177


<IPython.core.display.Javascript object>

In [12]:
response = sgmkr_clnt.create_endpoint_config(
    EndpointConfigName=ep_config_name,
    ProductionVariants=[
        {
            "VariantName": "version-1",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m4.xlarge",
            # sever_less = ''
        },
    ],
)

print(response)

{'EndpointConfigArn': 'arn:aws:sagemaker:ap-southeast-2:042990164028:endpoint-config/tmp-ep-config-2023-03-22-19-15-04-118177', 'ResponseMetadata': {'RequestId': '4d2e8005-d4fb-45d1-9113-f57bc6157274', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4d2e8005-d4fb-45d1-9113-f57bc6157274', 'content-type': 'application/x-amz-json-1.1', 'content-length': '126', 'date': 'Wed, 22 Mar 2023 08:15:04 GMT'}, 'RetryAttempts': 0}}


<IPython.core.display.Javascript object>

In [13]:
ep_name = "tmp-ep-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_name)

tmp-ep-2023-03-22-19-15-06-049247


<IPython.core.display.Javascript object>

In [14]:
response = sgmkr_clnt.create_endpoint(
    EndpointName=ep_name, EndpointConfigName=ep_config_name,
)
print(response)

{'EndpointArn': 'arn:aws:sagemaker:ap-southeast-2:042990164028:endpoint/tmp-ep-2023-03-22-19-15-06-049247', 'ResponseMetadata': {'RequestId': '5414cf48-20bf-45c1-9c6a-7a907267fe4f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5414cf48-20bf-45c1-9c6a-7a907267fe4f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '106', 'date': 'Wed, 22 Mar 2023 08:15:06 GMT'}, 'RetryAttempts': 0}}


<IPython.core.display.Javascript object>

In [15]:
waiter = sgmkr_clnt.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=ep_name, WaiterConfig={"Delay": 123, "MaxAttempts": 123})
print("Endpoint created")

Endpoint created


<IPython.core.display.Javascript object>

In [16]:
payload = "7.7, 3.0, 6.1, 2.3"
# payload = '7.7, 3.0, 6.1, 2.3 \n 7.9, 3.8, 6.4, 2.1'

<IPython.core.display.Javascript object>

In [20]:
sgmkr_runt = boto3.client("runtime.sagemaker")

<IPython.core.display.Javascript object>

In [22]:
response = sgmkr_runt.invoke_endpoint(
    EndpointName=ep_name, ContentType="text/csv", Body=payload,
)

prediction = response["Body"].read().decode()
print(prediction)

2.0


<IPython.core.display.Javascript object>

In [ ]:
print("Deleting sagemaker endpoint")
response = sgmkr_clnt.delete_endpoint(EndpointName = endpoint_name)  
print("Deleted sagemaker endpoint")
#print(response)    

In [ ]:
print("Deleting sagemaker endpoint configuration")
response = sgmkr_clnt.delete_endpoint_config(EndpointConfigName = endpoint_config_name)
print("Deleted sagemaker endpoint configuration")
#print(response)